In [4]:
import cv2

In [12]:
#'first_frame' variable will store 1st frame from 'video', will be used to compare subsequent frames.
first_frame = None

#captures video from built-in camera. (pass arg. '1' for webcam & so on)
video = cv2.VideoCapture(0)

#while loop will execute frame by frame
while True:
    #video.read() will return 2 values (a bool if gets frame {check} & a 3d array of image {frame})
    check, frame = video.read()
    
    #converting frames to grayscale
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #converting grayscale image to blured
    blured_img = cv2.GaussianBlur(gray_img, (21,21), 0 )
    
    #saving the 1st frame for comparision
    if first_frame is None:
        first_frame = blured_img
        
    #comparing & finding difference b/w 1st frame & current frame
    delta_frame = cv2.absdiff(first_frame, blured_img)
    
    #provide a threshold value, if pixel difference > 30 then convert it to white {255}, else black
    thresh_delta = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta, None, iterations=0 )
    
    #finding contours/outlines/borders of detected object
    (contours,_) = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #eliminating small objects (fly, noise) from making rectangle on them
    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue
        
        #getting x, y cooedinates, width & height of contours (outlines) for making rectangle on them
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #making rectangles (x, y coordinates starts from top-left (0,0)), giving green color {0,255,0} & 3 channels (RGB)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)
     
    #displaying 'frame' = original video + rectangles), 'Capturing' = grayscale video, 'delta' = differences & 'thresh' = displaying differences in Black and White
    cv2.imshow('frame', frame)
    cv2.imshow('Capturing', gray_img)
    cv2.imshow('delta', delta_frame)
    cv2.imshow('thresh', thresh_delta)
    
    #frame will change after 1 millisecond
    key = cv2.waitKey(1)
    
    #will break the while loop on pressing 'q'
    if key == ord('q'):
        break

video.release()

#will close all windows (executes after while loop)
cv2.destroyAllWindows()